The goal of this notebook is to use the re-created code from the old Repo to get the same graph.

It'll be taking the original implementations and tweaking them to get it working

In [ ]:
import json
import torch
from tqdm import tqdm
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

import matplotlib.pyplot as plt

from repeng import DatasetEntry
from repeng.reading_pipeline import RepReading


/workspace/repeng/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available()else "mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='cuda')

In [3]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left", legacy=False) # Wonder what padding left does
tokenizer.pad_token_id = 0 

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


In [4]:
rep_token = -1
hidden_layers = list(range(-1, -model.config.num_hidden_layers, -1))
n_difference = 1

# TODO: replicate this pipeline implementation
# rep_reading_pipeline =  pipeline("rep-reading", model=model, tokenizer=tokenizer)

In [ ]:
# Directly taken from new code - not using old dataset at all. Consider trying out old dataset if this is a problem
user_tag, asst_tag = "[INST]", "[/INST]"
with open("data/true_facts.json") as f:
    suffixes = json.load(f)

# see the emotion notebook for more details about this
positive_personas = ["honest"]
negative_personas = ["untruthful"]


def template(persona: str, suffix: str) -> str:
    return f"{user_tag} Pretend you're an {persona} person making statements about the world. {asst_tag} {suffix}"


dataset = []
for suffix in suffixes:
    tokens = tokenizer.tokenize(suffix)
    for i in range(1, len(tokens) - 5):
        truncated = tokenizer.convert_tokens_to_string(tokens[:i])
        for positive_persona, negative_persona in zip(
            positive_personas, negative_personas
        ):
            dataset.append(
                DatasetEntry(
                    positive=template(positive_persona, truncated),
                    negative=template(negative_persona, truncated),
                )
            )

# print some example entries
for i in range(3):
    print(f"dataset[{i}].positive:", dataset[i].positive)
    print(f"dataset[{i}].negative:", dataset[i].negative)

dataset[0].positive: [INST] Pretend you're an honest person making statements about the world. [/INST] The
dataset[0].negative: [INST] Pretend you're an untruthful person making statements about the world. [/INST] The
dataset[1].positive: [INST] Pretend you're an honest person making statements about the world. [/INST] The Earth
dataset[1].negative: [INST] Pretend you're an untruthful person making statements about the world. [/INST] The Earth
dataset[2].positive: [INST] Pretend you're an honest person making statements about the world. [/INST] The Earth'
dataset[2].negative: [INST] Pretend you're an untruthful person making statements about the world. [/INST] The Earth'


In [ ]:
# First we get directions
# honesty_rep_reader = rep_reading_pipeline.get_directions(
#     dataset['train']['data'], 
#     rep_token=rep_token, 
#     hidden_layers=hidden_layers, 
#     n_difference=n_difference, 
#     train_labels=dataset['train']['labels'], 
#     direction_method=direction_method,
#     batch_size=32,
# )
pipeline = RepReading(model, tokenizer)
rep_reader = pipeline.get_directions(dataset)
